In [1]:
!pip install -U langchain-chroma
!pip install sentence-transformers

  Using cached tokenizers-0.20.3-cp310-none-win_amd64.whl.metadata (6.9 kB)
Using cached tokenizers-0.20.3-cp310-none-win_amd64.whl (2.4 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.21.0
    Uninstalling tokenizers-0.21.0:
      Successfully uninstalled tokenizers-0.21.0


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
transformers 4.47.1 requires tokenizers<0.22,>=0.21, but you have tokenizers 0.20.3 which is incompatible.


  Using cached tokenizers-0.21.0-cp39-abi3-win_amd64.whl.metadata (6.9 kB)
Using cached tokenizers-0.21.0-cp39-abi3-win_amd64.whl (2.4 MB)
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3


ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
chromadb 0.5.23 requires tokenizers<=0.20.3,>=0.13.2, but you have tokenizers 0.21.0 which is incompatible.


In [2]:
# Langchain dependencies
from langchain_chroma import Chroma
from langchain.document_loaders.pdf import PyPDFDirectoryLoader # Importing PDF loader from Langchain
from langchain.text_splitter import RecursiveCharacterTextSplitter # Importing text splitter from Langchain
from langchain.schema import Document # Importing Document schema from Langchain
from langchain_chroma import Chroma # Importing Chroma vector store from Langchain
import os # Importing os module for operating system functionalities
import shutil # Importing shutil module for high-level file operations
import re

0. 불필요한 부분 제거

In [3]:
def clean_text(text):
    """Clean the raw text by removing unnecessary spaces, newlines, and special characters."""
    text = text.strip()  # Remove leading and trailing whitespace
    text = re.sub(r'\s+', ' ', text)  # Replace multiple spaces with a single space
    text = re.sub(r'(\n|\r)', ' ', text)  # Remove newlines
    text = re.sub(r'[^\w\s]', '', text)  # Remove special characters
    return text

1. 벡터화할 파일 로드하기

In [4]:
# Directory to your pdf files:
DATA_PATH = r'C:\Users\bhks0\Desktop\LLM 응용1\Document'
def load_documents():
    document_loader = PyPDFDirectoryLoader(DATA_PATH)  # PDF 파일 로드
    raw_documents = document_loader.load()  # 원본 문서 로드
    
    # 각 문서를 전처리
    cleaned_documents = []
    for doc in raw_documents:
        cleaned_text = clean_text(doc.page_content)  # clean_text 호출
        doc.page_content = cleaned_text  # 문서의 내용을 전처리된 텍스트로 업데이트
        cleaned_documents.append(doc)
    
    print(f"Loaded and cleaned {len(cleaned_documents)} documents.")
    return cleaned_documents

2. 문서 Split

In [5]:
def split_text(documents: list[Document]):
  # Initialize text splitter with specified parameters
  text_splitter = RecursiveCharacterTextSplitter(
    chunk_size=300, # Size of each chunk in characters
    chunk_overlap=100, # Overlap between consecutive chunks
    length_function=len, # Function to compute the length of the text
    add_start_index=True, # Flag to add start index to each chunk
  )

  # Split documents into smaller chunks using text splitter
  chunks = text_splitter.split_documents(documents)
  print(f"Split {len(documents)} documents into {len(chunks)} chunks.")

  # Print example of page content and metadata for a chunk
  document = chunks[0]
  print(document.page_content)
  print(document.metadata)

  return chunks # Return the list of split text chunks

3. Chroma DB에 저장

In [6]:
# Path to the directory to save Chroma database
from langchain.embeddings import HuggingFaceEmbeddings

CHROMA_PATH = './chroma'
modelPath = "sentence-transformers/paraphrase-multilingual-mpnet-base-v2" # open source embedding model

# Create a dictionary with model configuration options, specifying to use the CPU for computations
model_kwargs = {'device': 'cpu'}

# Create a dictionary with encoding options, specifically setting 'normalize_embeddings' to False
encode_kwargs = {'normalize_embeddings': False}

# Initialize an instance of HuggingFaceEmbeddings with the specified parameters
embeddings = HuggingFaceEmbeddings(
    model_name=modelPath,     # Provide the pre-trained model's path
    model_kwargs=model_kwargs, # Pass the model configuration options
    encode_kwargs=encode_kwargs # Pass the encoding options
)

def save_to_chroma(chunks: list[Document]):
  # Clear out the existing database directory if it exists
  if os.path.exists(CHROMA_PATH):
    shutil.rmtree(CHROMA_PATH)

  # Create a new Chroma database from the documents using OpenAI embeddings
  db = Chroma.from_documents(
    chunks,
    embeddings,
    persist_directory=CHROMA_PATH
  )

  # Persist the database to disk
  print(f"Saved {len(chunks)} chunks to {CHROMA_PATH}.")

C:\Users\bhks0\AppData\Local\Temp\ipykernel_27356\872137747.py:14: LangChainDeprecationWarning: The class `HuggingFaceEmbeddings` was deprecated in LangChain 0.2.2 and will be removed in 1.0. An updated version of the class exists in the :class:`~langchain-huggingface package and should be used instead. To use it run `pip install -U :class:`~langchain-huggingface` and import as `from :class:`~langchain_huggingface import HuggingFaceEmbeddings``.
  embeddings = HuggingFaceEmbeddings(
c:\Users\bhks0\anaconda3\envs\eunbin\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [7]:
def generate_data_store():
  documents = load_documents() # Load documents from a source
  chunks = split_text(documents) # Split documents into manageable chunks
  save_to_chroma(chunks) # Save the processed data to a data store

# Load environment variables from a .env file
#load_dotenv()
# Generate the data store

generate_data_store()

Loaded and cleaned 188 documents.
Split 188 documents into 1105 chunks.
반갑습니다 어떤 일이 있으신 건가요 제가 최대한 도와드릴게요 지금 매우 어려운 상황을 겪고 있군요 혹시 이 상황이 어떻게 시작됐나요 갑작스럽게 스트레스를 느끼고 계시는 것 같군요 어떻게 하면 스트레스를 해소하실 수 있을지 함께 생각해보아요 물론 가능해요 스트레스를 해소하는 방법 중에 가장 효과적인 것은 일단 그 원인을 파악하는 것이에요 지금 학교 가족 친구 등에서 스트레스를 받고 있으신 것 같아요 각각의 상황들에 대해서 자세히 얘기해보실래요 학교에서의 압박감과 공부 가족관계 친구관계 등 다양한 요인들로 인해 어려움을 겪고 있는 것
{'source': 'C:\\Users\\bhks0\\Desktop\\LLM 응용1\\Document\\멀티턴심리상담.pdf', 'page': 0, 'start_index': 0}
Saved 1105 chunks to ./chroma.
